## Auditing and Exporting Detections

#### Before running this script, make sure that your Google Drive folder contains the orthomosaic GeoTiff (`step 0`) and no other GeoTIFF files, the `data.json` file (`step 1`), the `classes.csv` file (`step3`) and the `new_detections.json` file (`step 4`). If you want to add your new CNN detections to the manually annotated detections you created in VIA, also add the JSON file you exported using VIA (`step 2`). You will need to input that file name directly, since it is not standardized in our workflow.

<a href="https://colab.research.google.com/github/gl7176/GreySealCNN/blob/master/5_export_detections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
#####  <center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

### Connect to our Google Drive folder and pull annotation CSV

In [1]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D'

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
orthomosaic_file = {}
for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  if fname.endswith(".tif") or fname.endswith(".json") or fname.endswith("classes.csv"):
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    os.stat(fname)
    # if the file is a *.tif and larger than 100 mb we label it the orthomosaic
    if fname.endswith(".tif") and os.stat(fname).st_size > 10^8 :
      # if there are multiple orthomosaic files detected we spit an error
      if len(orthomosaic_file) != 0:
        raise Exception("more than one orthomosaic file identified based on size and type")
      orthomosaic_file = fname
      print("orthomosaic identified as " + orthomosaic_file)
print("all files pulled")

orthomosaic identified as data/Hay Island 2015.tif
all files pulled


### Set up the python environment and key variables

In [1]:
import os
import argparse
import numpy as np
import json
import csv
!pip install rasterio
import rasterio

!shapely.geometry import mapping, Polygon
!pip install fiona
import fiona # only required for exporting to shapefiles

/bin/bash: shapely.geometry: command not found


### Exporting to Shapefile

In [128]:
# update box locations from local tile coordinates to orthomosaic coordinates
# using image_locations from original tiling process, invoked by filename
# ---this involves transforming x1, y1, x2, y2 coordinates to 4 sets of x,y points
# and then transforming them back to the original form---
# then build a list of detection dictionaries. Note that filename is no longer
# needed because coordinates are now relative to the orthomosaic, not tile

# open the output from our CNN
with open(local_download_path + '/new_detections.json') as f:
    detected_labels = json.load(f)

# open the output from our original tile splitting
with open(local_download_path + '/data.json') as f:
  img_data = json.load(f)

detection_list = []
for key, value in detected_labels.items():
    for detection in value:
      # convert bounding box from x1/y1/x2/y2 format to [x1,y1],[x2,y1],[x2,y2],[x2,y1] coordinates format 
      bounding_box = np.array([[detection['box'][0], detection['box'][1]], [detection['box'][2], detection['box'][1]], [detection['box'][2], detection['box'][3]], [detection['box'][0], detection['box'][3]]])
      # update the new coordinates format from local tile coordinates to orthomosaic coordinates
      bounding_box = bounding_box + [img_data["image_locations"][[key][0].split("/")[-1]]]
      # convert our bounding box from coordinates format back to x1/y1/x2/y2 format
      bounding_box = [bounding_box[0][0], bounding_box[0][1], bounding_box[1][0], bounding_box[2][1]] 
      # update the dictionary
      detection['box'] = bounding_box
      #build our detection list. Note we no longer need filenames because the coordinates are no longer local
      detection_list.append(detection)
print(detection_list[0:3])

[{'box': [8066, 4623, 8112, 4655], 'label': 1, 'score': 0.8100398778915405}, {'box': [8042, 4641, 8079, 4677], 'label': 1, 'score': 0.796779215335846}, {'box': [8037, 5080, 8086, 5156], 'label': 0, 'score': 0.7938922047615051}]


In [129]:
# pull labels from classes.csv
import csv
with open(local_download_path + '/classes.csv', "r") as f:
    reader = csv.reader(f, delimiter=",")
    labels_to_names = {int(i[1]):i[0] for i in reader}

boxes = []
scores = []
for detection in detection_list:
  boxes.append(detection['box'])
  scores.append(detection['score'])

In [130]:
# Malisiewicz et al.
# https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
# import the necessary packages
import numpy as np

def non_max_suppression(boxes, probs, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)

    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return the index of the bounding boxes that were picked
    return pick

In [131]:
bboxes = np.array(boxes)
pick = non_max_suppression(bboxes, scores, 0.6)
nms_detection_list = []
for i in pick:
  nms_detection_list.append(detection_list[i])
print("Before NMS: " + str(len(detection_list)) + " files")
print("After NMS: " + str(len(nms_detection_list)) + " files")

Before NMS: 132 files
After NMS: 126 files


In [ ]:
# if you run this section multiple times it will repeatedly (and incorrectly) transform your coordinates. Don't do it.
geolocated_bb = []
for detection in nms_detection_list:
  bounding_box = np.array([[detection['box'][0], detection['box'][1]], [detection['box'][2], detection['box'][1]], [detection['box'][2], detection['box'][3]], [detection['box'][0], detection['box'][3]]]).astype(float)
  count = 0
  for point in bounding_box:
    point = dataset.transform * point
    bounding_box[count] = point
    count += 1
  detection['box'] = bounding_box
print(nms_detection_list)

In [134]:
print(nms_detection_list[0:3])

[{'box': array([[ 292130.18141, 5099948.34892],
       [ 292131.7264 , 5099948.34892],
       [ 292131.7264 , 5099947.16323],
       [ 292130.18141, 5099947.16323]]), 'label': 1, 'score': 0.9102174639701843}, {'box': array([[ 292140.3496 , 5099960.16989],
       [ 292142.28982, 5099960.16989],
       [ 292142.28982, 5099957.29549],
       [ 292140.3496 , 5099957.29549]]), 'label': 0, 'score': 0.887717604637146}, {'box': array([[ 292103.44949, 5100016.40034],
       [ 292106.00052, 5100016.40034],
       [ 292106.00052, 5100014.10082],
       [ 292103.44949, 5100014.10082]]), 'label': 0, 'score': 0.8841370940208435}]


In [138]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Set output directory
output_dir = 'shapefile_output'

# create the dir if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str')
  ])
}

with fiona.open(output_dir + '/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(nms_detection_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', orthomosaic_file),
                                       ('Detection', labels_to_names[polygon['label']])
                                       ]),
            'type': 'Feature'}
      c.write(record)

### Zip output folder for download

In [136]:
# zip up the output directory into an archive for download
import subprocess
subprocess.call(['zip', '-r', '/content/' + output_dir + '.zip', '/content/' + output_dir])

from google.colab import files
files.download("/content/" + output_dir + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### RetinaNet to Existing VIA

In [ ]:
# open the previous training data

with open(local_download_path + '/via_SealCNN_TrainingData.json') as f:
    existing_labels = json.load(f)

# add the new detections to the old via_region_data.json file

#"11_fiX1mEhK","[""2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png""]",0,"[]","[2,509.275,929.174,89.376,48.904]","{}"

for filepath, detections in detected_labels.items():
    fn = filepath.split("/")[-1]
    # TODO is deep copy correct?
    for filename_size, metadata in existing_labels.items():
        if fn == metadata["filename"]:
            for detection in detections:
                # 'box' : [x1, y1, x2, y2]
                x1 = detection["box"][0]
                y1 = detection["box"][1]
                x2 = detection["box"][2]
                y2 = detection["box"][3]
                #print(x1,x2,y1,y2)
                metadata["regions"].append({'shape_attributes': {'name': 'rect', 'x': x1, 'y': y1, 'width': x2-x1, 'height': y2-y1}, 'region_attributes': {}})
    

In [ ]:
# write out new VIA file with additional detections

with open(local_download_path + '/via_region_data_detections.json', 'w') as fp:
    json.dump(existing_labels, fp)